## Random forest

#### Read and Clean the data 

In [1]:
import nltk
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t', header = None)
data.columns = ['label', 'body_text']


def count_punc(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text)-text.count(" ")), 3)* 100

data['body_len'] = data['body_text'].apply(lambda x: len(x)-x.count(" "))
data['punc%'] = data['body_text'].apply(lambda x: count_punc(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punc%'], pd.DataFrame(X_tfidf.toarray())], axis =1)
X_features.head()




,body_len,punc%,0,1,2,3,4,5,6,7,...,8099,8100,8101,8102,8103,8104,8105,8106,8107,8108
0,92,9.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,24,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,39,15.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Explore RandomforestClassifier through Holdout Set

In [3]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size = 0.2)


In [5]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=50, max_depth =  20, n_jobs=-1)
rf_model = rf.fit(X_train, y_train)

In [8]:
sorted(zip(rf_model.feature_importances_, X_train.columns), reverse=True)[0:10]

[(0.0816878557051253, 'body_len'),
 (0.04981307433196248, 1805),
 (0.03600142768822721, 6290),
 (0.03496395772454048, 2033),
 (0.024159709724072487, 3136),
 (0.022876940628895212, 5729),
 (0.020115925799407356, 6751),
 (0.01877145755847646, 3447),
 (0.018563389889233485, 4801),
 (0.018276988876119837, 5993)]

In [9]:
y_pred = rf_model.predict(X_test)
precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')


In [11]:
print('Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision, 3),
                                                        round(recall, 3),
                                                        (y_pred==y_test).sum()/len(y_pred)))

Precision: 1.0 / Recall: 0.679 / Accuracy: 0.9605381165919282
